In [1]:
import tflscripts
from tflscripts import take_percentage_of_data, read_dataset, filter_by_features, X_sort, filter_by_activities_transfer, classify, easy_domain_adaptation_update_dataframes, take_multiple_percentages_of_data, concat_and_reindex, filter_by_activities, read_and_filter_dataset, build_pipeline
from sklearn.metrics import accuracy_score
import numpy as np
from scipy.stats import mode
import pandas as pd
from sklearn.preprocessing import LabelEncoder
from itertools import combinations

In [3]:
configuration = tflscripts.read_configuration()
use_features = 'microphone.*index_mass_quantile'
force_columns = None
use_columns = None
use_activities = [configuration['activities'].index(a) for a in configuration['activity_sets'][0]]

training_source_data_ratio = 0.6
training_target_data_ratio = 0.6
testing_target_data_ratio = 0.4

tflscripts.set_dataset_folder('/home/giotto/transfer-learning-playground/datasets/')

In [4]:
df_s1, df_s1_labels = read_and_filter_dataset(
        'robotics-final',
        'xdk_1',
        use_features=use_features,
        force_columns=force_columns,
        use_columns=use_columns,
        use_activities=use_activities,
        scale=True,
        with_feature_selection=False)

df_s2, df_s2_labels = read_and_filter_dataset(
        'synergy-final-iter1',
        'xdk_1',
        use_features=use_features,
        force_columns=force_columns,
        use_columns=use_columns,
        use_activities=use_activities,
        scale=True,
        with_feature_selection=False)

df_s1_labels_adapted = df_s1_labels.copy()
df_s1_labels_adapted.label = 0

df_s2_labels_adapted = df_s2_labels.copy()
df_s2_labels_adapted.label = 1

df_s1_s2, df_s1_s2_labels = concat_and_reindex(
        [df_s1, df_s2],
        [df_s1_labels_adapted, df_s2_labels_adapted])

clf_ppl = build_pipeline(df_s1_s2, df_s1_s2_labels, scale=True)

ppl_s1 = build_pipeline(df_s1, df_s1_labels, scale=True)
ppl_s2 = build_pipeline(df_s2, df_s2_labels, scale=True)

df_target, df_target_labels = read_and_filter_dataset(
        'scott-final-iter1',
        'xdk_1',
        use_features=use_features,
        force_columns=force_columns,
        use_columns=use_columns,
        use_activities=use_activities,
        scale=True,
        with_feature_selection=False)

clfs = clf_ppl.predict(df_target)
target_values = df_target.values
pred = [ppl_s1.predict([target_values[i]])[0] if c == 0 else ppl_s2.predict([target_values[i]])[0] for i, c in enumerate(clfs)]

accuracy_score(df_target_labels['label'], pred)

0.45595854922279794

In [5]:
accuracy_score(df_target_labels['label'], ppl_s1.predict(df_target))

0.4689119170984456

In [6]:
accuracy_score(df_target_labels['label'], ppl_s2.predict(df_target))

0.43005181347150256